In [2]:
import numpy as np
import pandas as pd
from pandas import json_normalize
import seaborn as sns
import matplotlib.pyplot as plt

import time
import datetime

import json
import os
from dotenv import load_dotenv #pip install python-dotenv

from bs4 import BeautifulSoup
import requests 

from selenium import webdriver 
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import warnings
warnings.filterwarnings("ignore")

In [6]:
# comprovar quantes requests he fet i quantes me'n queden

url_search = "https://api.discogs.com/database/search"

params = {
    "type": "release"  # Search for masters
    , "format": "album"           # just search for albums
    , "per_page": 100    # Number of results per page (max 100)
    , "page": 1          # Start with page 1
    , 'country': 'UK'
    , "year": 2016
    , 'genre': 'Rock'
    # , "style": style
}

headers = {"User-Agent": "MyDiscogsApp/1.0", "Authorization": "Discogs token=UwfqmsztxwnfABgQpmhaAsprbUgpOJKGOJSQAqfp"}
response = requests.get(url_search, headers=headers, params=params)

rate_limit = response.headers.get("X-Discogs-Ratelimit")
rate_limit_used = response.headers.get("X-Discogs-Ratelimit-Used")
rate_limit_remaining = response.headers.get("X-Discogs-Ratelimit-Remaining")

print(f"Rate Limit: {rate_limit}")
print(f"Rate Limit Used: {rate_limit_used}")
print(f"Rate Limit Remaining: {rate_limit_remaining}")

ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'Se ha forzado la interrupción de una conexión existente por el host remoto', None, 10054, None))

### **Proves**: un any en un país

In [13]:
# get all rock_styles in one year in one country

url = "https://api.discogs.com/database/search"
headers = {"Authorization": "Discogs token=UwfqmsztxwnfABgQpmhaAsprbUgpOJKGOJSQAqfp"}

genres = ["Blues", "Classical", "Electronic", "Folk, World, & Country", "Funk / Soul", "Hip-Hop", "Jazz", "Latin", "Pop", "Rock", "Stage & Screen"]
rock_styles = ["Pop Rock", "Punk", "Alternative Rock", "Indie Rock", "Hardcore", "Rock & Roll", "Hard Rock", "Psychedelic Rock"
              , "Black Metal", "Heavy Metal", "Folk Rock", "Experimental", "Prog Rock", "Death Metal", "Garage Rock"]

for style in rock_styles:
    # Define parameters for artist search
    params = {
        "per_page": 100    # Number of results per page (max 100)
        , 'type': 'release'
        , 'format': 'album'
        , 'genre': 'Rock'

        # items to change
        , 'country': 'UK'
        , 'year': 2016
        , 'style': style
    }

    response = requests.get(url, headers=headers, params=params)
    data = response.json()
    items_number = data['pagination']['items']  # number of items

    print(f"{style}: {items_number}")

Pop Rock: 531
Punk: 973
Alternative Rock: 822
Indie Rock: 1053
Hardcore: 256
Rock & Roll: 115


ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'Se ha forzado la interrupción de una conexión existente por el host remoto', None, 10054, None))

In [15]:
# get one style in one year in one country

url = "https://api.discogs.com/database/search"
headers = {"User-Agent": "MyDiscogsApp/1.0", "Authorization": "Discogs token=UwfqmsztxwnfABgQpmhaAsprbUgpOJKGOJSQAqfp"}

# Define parameters for artist search
params = {
    "per_page": 100    # Number of results per page (max 100)
    , 'type': 'release'
    , 'format': 'album'
    , 'page': 1

    # items to change
    , 'country': 'UK'
    , 'year': 2023
    , 'genre': 'Rock'
    # , 'style': 'Alternative Rock'
}

response = requests.get(url, headers=headers, params=params)
data = response.json()
items_number = response.json()['pagination']['items']  # number of items

print(f"Items: {items_number}")

# response.json()

Items: 5382


### **Rock masters from a specific country**

In [3]:
df_masters_scrapped = pd.read_csv('Datasets/df_masters.csv')
df_masters_scrapped.shape

(10870, 13)

In [8]:
# rock_styles = ["Pop Rock", "Punk", "Alternative Rock", "Indie Rock", "Hardcore", "Rock & Roll",

# Styles scrapped of 2016

In [7]:
url_search = "https://api.discogs.com/database/search"
headers = {"User-Agent": "MyDiscogsApp", "Authorization": "Discogs token=UwfqmsztxwnfABgQpmhaAsprbUgpOJKGOJSQAqfp"}

df_masters_scrapped = pd.read_csv('Datasets/df_masters.csv')

# rock_styles = ["Pop Rock", "Punk", "Alternative Rock", "Indie Rock", "Hardcore", "Rock & Roll", "Hard Rock", "Psychedelic Rock"
#               , "Black Metal", "Heavy Metal", "Folk Rock", "Experimental", "Prog Rock", "Death Metal", "Garage Rock"]

rock_styles = ["Hard Rock", "Psychedelic Rock"
              , "Black Metal", "Heavy Metal", "Folk Rock", "Experimental", "Prog Rock", "Death Metal", "Garage Rock"]

# create empty lists
artist_id_list = []
artists_list = []
titles_list = []
years_list = []
release_countries_list = []
album_length_list = []
tracks_list = []
release_type_list = []
genres_list = []
styles_list = []
master_id_new_list = []
artist_profile_list = []
main_release_id_list = []

attempts = 0
scrapped = 0
start_year = 2016

for year in range(start_year, start_year-1, -1):
    print(f'Year: {year}')

    for style in rock_styles:
        print(f'Style: {style}')

        # Define parameters for album search
        params = {
            "type": "release"  # Search for masters
            , "format": "album"           # just search for albums
            , "per_page": 100    # Number of results per page (max 100)
            , "page": 1          # Start with page 1
            , 'country': 'UK'
            , "year": year
            , 'genre': 'Rock'
            , "style": style
        }

        response1 = requests.get(url_search, headers=headers, params=params)
        time.sleep(1)
        data1 = response1.json()

        items = data1['pagination']['items']
        print(f'Items: {items} releases')

        pages = data1['pagination']['pages']+1
        
        for page in range(1, pages):
            print(f'------------ Page: {page} of {pages-1}, {style} of {year} -------------')

            params['page'] = page     # change pages
            response2 = requests.get(url_search, headers=headers, params=params)
            data2 = response2.json()

            for id in range(len(data2['results'])):
                attempts+=1
                time.sleep(1)

                master_id = data2['results'][id]['master_id']

                if master_id == 0:
                    print(f'{attempts} attempts - {scrapped} scrapped - {((page-1)*100)+id+1}/{items} - master_id {master_id}')
                    continue        # I cannot get info 
                elif master_id in master_id_new_list:
                    print(f'{attempts} attempts - {scrapped} scrapped - {((page-1)*100)+id+1}/{items} - master_id {master_id} already in list') 
                    continue        # there are several releases of the same master_release, I just want one (the master_release)
                elif master_id in df_masters_scrapped['master_id'].values:
                    print(f'{attempts} attempts - {scrapped} scrapped - {((page-1)*100)+id+1}/{items} - master_id {master_id} already scrapped') 
                    continue        # I have already scrapped it before
                else:
                    url_master = f"https://api.discogs.com/masters/{master_id}"
                    response3 = requests.get(url_master, headers=headers)
                    time.sleep(1)
                    data3 = response3.json()

                    artist = data3['artists'][0]['name']

                    if artist == 'Various':
                            print(f'{attempts} attempts - {scrapped} scrapped - {((page-1)*100)+id+1}/{items} - master_id {master_id}: Compilation') 
                            continue   # I don't want compilations
                    else:
                    # define the data I want to get
                        title = data3['title']
                        release_year = data3['year']                 
                        genres = data3['genres']
                        styles = data3['styles']

                    # a more complex information is the total duration of the album, I have to sum the durations of each song
                        number_of_songs = len(data3['tracklist'])
                        song_durations = []

                        for song in range(number_of_songs):
                            song_duration = data3['tracklist'][song]['duration']
                            try:
                                minutes = int(song_duration.split(':')[0])
                                seconds = int(song_duration.split(':')[1])
                                song_duration_minutes = minutes + seconds/60
                                song_durations.append(song_duration_minutes)
                            except:
                                pass
                            album_length = round(sum(i for i in song_durations), 2)

                        main_release_id = data3['main_release']
                        artist_id = data3['artists'][0]['id']

                    # release page
                        url = f"https://api.discogs.com/releases/{main_release_id}"
                        time.sleep(1)
                        response4 = requests.get(url, headers=headers)
                        data4 = response4.json()

                    # define the data I want to get
                        release_country = data4['country']
                        release_type = data4['formats'][0]['descriptions']

                    # artist page
                        url = f"https://api.discogs.com/artists/{artist_id}"
                        time.sleep(1)
                        response5 = requests.get(url, headers=headers)
                        data5 = response5.json()

                    # define the data I want to get
                        try:
                            artist_profile = data5['profile']
                        except:
                            artist_profile = np.nan
                            pass

                    # save the info in the lists
                        artist_id_list.append(str(artist_id))
                        artists_list.append(artist)
                        titles_list.append(title)
                        years_list.append(release_year)
                        genres_list.append(genres)
                        styles_list.append(styles)
                        master_id_new_list.append(master_id)
                        album_length_list.append(album_length)
                        tracks_list.append(number_of_songs)
                        release_countries_list.append(release_country)
                        release_type_list.append(release_type)
                        main_release_id_list.append(main_release_id)

                        if artist_profile == '':
                            artist_profile_list.append(np.nan)
                        else:
                            artist_profile_list.append(artist_profile)

                        scrapped+=1

                    # list lengths
                        lists = [artist_id_list, main_release_id_list, master_id_new_list, release_countries_list, artists_list, titles_list, album_length_list
                                 , tracks_list, release_type_list, genres_list, styles_list, artist_profile_list, years_list]

                    # Check if all lists have the same length
                        lengths = [len(lst) for lst in lists]
                        if len(set(lengths)) != 1:
                            print("Lengths are not the same.")
                            break # stop the loop, I won't be able to store the data if I have one value missing

                        print(f'{attempts} attempts - {scrapped} scrapped - {((page-1)*100)+id+1}/{items} - master_id {master_id}: {artist} - {title}')
                        time.sleep(1)

Year: 2016
Style: Hard Rock
Items: 277 releases
------------ Page: 1 of 3, Hard Rock of 2016 -------------


ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'Se ha forzado la interrupción de una conexión existente por el host remoto', None, 10054, None))

In [47]:
# UK 2016

# Pop Rock: 531
# Punk: 973
# Alternative Rock: 822
# Indie Rock: 1053
# Hardcore: 256
# Rock & Roll: 115
# Hard Rock: 277
# Psychedelic Rock: 707
# Black Metal: 138
# Heavy Metal: 213
# Folk Rock: 372
# Experimental: 489
# Prog Rock: 400
# Death Metal: 133
# Garage Rock: 193

277+707+138+213+372+489+400+133+193

2922

In [34]:
(167.5*60)/3889

2.584211879660581

### **Saving the data**

In [35]:
# list lengths
lists = [artist_id_list, master_id_new_list, release_countries_list, artists_list, titles_list, album_length_list, tracks_list , release_type_list, genres_list, styles_list, artist_profile_list, main_release_id_list]

# Check if all lists have the same length
lengths = [len(lst) for lst in lists]
if len(set(lengths)) == 1:
    print(len(master_id_new_list))
else:
    print("Lengths are not the same.")

# for lst in lists:
#     print(f'{len(lst)}') # check that all lists have the same length (even though I had made sure with the previous code)

1169


#### See which list is shorter than the rest

In [ ]:
lists = [
    ('master_id', master_id_new_list),
    ('release_country', release_countries_list),
    ('artists', artists_list),
    ('titles', titles_list),
    ('years', years_list),
    ('album_length', album_length_list),
    ('tracks', tracks_list),
    ('release_type', release_type_list),
    ('genres', genres_list),
    ('styles', styles_list),
    ('artist_profile', artist_profile_list)
]

# Check if all lists have the same length
lengths = [len(lst) for name, lst in lists]
if len(set(lengths)) == 1:
    print("All lists have the same length:", lengths[0])
else:
    print("Lengths are not the same.")

# for name, lst in lists:
#     print(f'{name}: {len(lst)}') # check that all lists have the same length (even though I had made sure with the previous code)

All lists have the same length: 35


#### Continue

In [36]:
df_masters = pd.DataFrame({'artist_id': artist_id_list
                        , 'master_id': master_id_new_list
                        , 'main_release_id': main_release_id_list
                        , 'release_country': release_countries_list
                        , 'artist': artists_list
                        , 'title': titles_list
                        , 'year': years_list
                        , 'album_length': album_length_list
                        , 'tracks': tracks_list
                        , 'release_type': release_type_list
                        , 'genres': genres_list
                        , 'styles': styles_list
                        , 'artist_profile': artist_profile_list
                        })
df_masters.shape

(1169, 13)

In [37]:
print(f'{df_masters.shape[0]} releases\n{df_masters.artist.nunique()} different artists')

1169 releases
1046 different artists


In [38]:
df_masters.sample(10)

,artist_id,master_id,main_release_id,release_country,artist,title,year,album_length,tracks,release_type,genres,styles,artist_profile
51,5253011,3217738,28059147,US,Citizen Way,2.0,2016,0.00,11,[Album],"[Rock, Pop]",[Pop Rock],Christian pop-rock group with a hip-hop sound.
1095,312415,287710,2771718,US,ELF (3),L.A. 59,1974,37.75,9,"[LP, Album]",[Rock],[Rock & Roll],Elf was a blues-rock band formed in upstate Ne...
844,4623125,1034869,8767278,UK,Terry (53),Terry HQ,2016,0.00,12,[LP],[Rock],"[Punk, Pop Rock, Post-Punk]","Indie rock band from Melbourne, Australia."
750,15259,971061,8120912,UK & Europe,Tindersticks,Philharmonie De Paris,2016,0.00,12,"[LP, Album, Limited Edition]",[Rock],"[Alternative Rock, Art Rock]",English band formed in 1991 out of [a275920] (...
822,3645870,1017238,8536196,UK,The Hummingbirds (3),Pieces Of You,2016,0.00,11,"[LP, Album]","[Rock, Blues]","[Electric Blues, Skiffle]",Liverpool 5 piece band.
518,2544241,983043,8345823,UK & US,Parquet Courts,Human Performance,2016,42.33,13,"[LP, Album, Deluxe Edition, Numbered]",[Rock],"[Punk, Indie Rock, Post-Punk, Art Rock]","noise folk from NYC, USA"
401,327875,1082235,5099993,UK,Police Bastard,Confined,2013,0.00,10,[Album],[Rock],"[Punk, Crust]",Birmingham based crust punk/hardcore crossover...
227,4361817,988624,8296482,UK & Europe,MOTHXR,Centerfold,2016,83.93,22,"[LP, Album]","[Electronic, Rock, Funk / Soul]","[Alternative Rock, Indie Rock, Contemporary R&...",Mothxr is a band from Brooklyn. Band members i...
423,4317117,1028660,8773623,UK,Happy Accidents (3),You Might Be Right,2016,0.00,11,"[LP, Album]",[Rock],"[Indie Rock, Punk]",Alternative rock band from UK.\r\nFormed 2015\...
499,1004520,1057609,9030536,US,All Time Low,"Straight To DVD 2: Past, Present, and Future H...",2016,78.82,25,[Album],[Rock],[],"Alternative rock band from Maryland, US.\r\nFo..."


In [39]:
df_masters.release_country.value_counts().head()

release_country
UK                 623
US                 219
Europe              74
UK & Europe         71
UK, Europe & US     26
Name: count, dtype: int64

In [40]:
df_masters.year.value_counts().head()

year
2016    719
2015     44
2017     18
2012     17
1986     16
Name: count, dtype: int64

### **Merge with the previous df**

In [41]:
# import the last df
df_masters_scrapped = pd.read_csv('Datasets/df_masters.csv')
df_masters_scrapped.shape

(9701, 13)

In [42]:
# concat with the df of the albums I just scrapped
df_masters_concat = pd.concat([df_masters_scrapped, df_masters])
df_masters_concat.reset_index(drop=True, inplace=True)
df_masters_concat.shape

(10870, 13)

In [43]:
def check_duplicates(df):
    if df['master_id'].duplicated().sum() == 0:
        print('No duplicates')
    else: 
        duplicates = df['master_id'].duplicated().sum()
        df.drop_duplicates(subset='master_id', inplace=True)
        df.reset_index(drop=True, inplace=True)
        print(f'Found {duplicates} duplicates')
        print(f'Resulting dataset: {df.shape}')

In [44]:
check_duplicates(df_masters_concat)

No duplicates


In [45]:
print(f'{df_masters_concat.shape[0]} releases\n{df_masters_concat.artist.nunique()} different artists')

10870 releases
7039 different artists


### **Exporting to csv**

In [46]:
df_masters_concat.to_csv('Datasets/df_masters.csv', index=False)

In [ ]:
# # import the last df
# df_masters_concat = pd.read_csv('Datasets/df_masters.csv')

### **Statistics**

In [30]:
df_masters_concat.year.value_counts().head(10)

year
2020    1251
2021    1081
2022    1038
2023     976
2024     778
2014     571
2015     388
2019     196
2003     155
2013     114
Name: count, dtype: int64

In [31]:
df_masters_concat.release_country.value_counts().head(10)

release_country
UK                 5313
US                 1364
Europe              593
UK & Europe         427
UK, Europe & US     199
Germany             191
Worldwide           184
UK & US             180
Australia            96
Japan                88
Name: count, dtype: int64

In [54]:
df_masters_concat[df_masters_concat['artist']=='Genesis']

,artist_id,master_id,main_release_id,release_country,artist,title,year,album_length,tracks,release_type,genres,styles,artist_profile
1563,124506.0,46010,4774968,UK,Genesis,From Genesis To Revelation,1969,0.00,13,"['LP', 'Album', 'Mono']",['Rock'],"['Psychedelic Rock', 'Prog Rock']","Formed at Charterhouse School, Godalming, Surr..."
2285,124506.0,28853,715057,UK,Genesis,Foxtrot,1972,0.00,6,"['LP', 'Album']",['Rock'],['Prog Rock'],"Formed at Charterhouse School, Godalming, Surr..."
2295,124506.0,29256,4383526,UK,Genesis,The Lamb Lies Down On Broadway,1974,0.00,23,"['LP', 'Album', 'Stereo']",['Rock'],"['Prog Rock', 'Art Rock']","Formed at Charterhouse School, Godalming, Surr..."
2298,124506.0,29224,1702230,UK,Genesis,Selling England By The Pound,1973,0.00,8,"['LP', 'Album', 'Stereo']",['Rock'],['Prog Rock'],"Formed at Charterhouse School, Godalming, Surr..."
2301,124506.0,29373,14718304,UK,Genesis,Trespass,1970,0.00,6,"['LP', 'Album']",['Rock'],['Prog Rock'],"Formed at Charterhouse School, Godalming, Surr..."
2309,124506.0,28911,667372,UK,Genesis,Genesis,1983,0.00,9,"['LP', 'Album', 'Stereo']",['Rock'],"['Pop Rock', 'Art Rock']","Formed at Charterhouse School, Godalming, Surr..."
2313,124506.0,40691,1152731,UK,Genesis,Three Sides Live,1982,0.00,13,"['LP', 'Album', 'Stereo']",['Rock'],"['Pop Rock', 'Prog Rock']","Formed at Charterhouse School, Godalming, Surr..."
2323,124506.0,28832,773021,UK,Genesis,Duke,1980,54.77,12,"['LP', 'Album']",['Rock'],"['Pop Rock', 'Prog Rock']","Formed at Charterhouse School, Godalming, Surr..."
4766,124506.0,29002,512282,UK,Genesis,Invisible Touch,1986,0.00,8,"['LP', 'Album', 'Stereo']",['Rock'],"['Soft Rock', 'Pop Rock']","Formed at Charterhouse School, Godalming, Surr..."


In [49]:
df_masters_concat.artist.value_counts().head(10)

artist
Bob Dylan                             26
The Fall                              22
Six By Seven                          17
Uriah Heep                            14
Darren Lock                           14
The Bevis Frond                       13
Hawkwind                              13
King Gizzard And The Lizard Wizard    13
UFO (5)                               13
Black Sabbath                         12
Name: count, dtype: int64

In [52]:
df_masters_concat.genres.value_counts().head(10)

genres
['Rock']                              5826
['Electronic', 'Rock']                 590
['Rock', 'Pop']                        582
[Rock]                                 425
['Rock', 'Folk, World, & Country']     288
['Electronic', 'Rock', 'Pop']          129
[Rock, Pop]                             82
['Jazz', 'Rock']                        67
['Rock', 'Blues']                       67
[Electronic, Rock]                      63
Name: count, dtype: int64

In [53]:
df_masters_concat.styles.value_counts().head(10)

styles
['Indie Rock']                        491
['Punk']                              290
['Alternative Rock']                  278
['Black Metal']                       259
[]                                    228
['Psychedelic Rock']                  222
['Prog Rock']                         222
['Alternative Rock', 'Indie Rock']    157
['Death Metal']                       157
['Hard Rock']                         131
Name: count, dtype: int64

In [82]:
df_masters_concat[df_masters_concat['release_country']=='Europe'].sort_values('artist')

,artist_id,master_id,main_release_id,release_country,artist,title,year,album_length,tracks,release_type,genres,styles,artist_profile
1829,156155.0,118883,737915,Europe,3 Mustaphas 3,Shopping,1987,44.72,9,"['LP', 'Album']","['Rock', 'Folk, World, & Country']",['Folk Rock'],British band [b]3 Mustaphas 3[/b] did much to ...
4690,3750880.0,1706746,15028328,Europe,5 Seconds Of Summer,CALM,2020,39.50,12,['Album'],"['Rock', 'Pop']","['Pop Rock', 'Power Pop']",5 Seconds Of Summer (5SOS) is an Australian po...
1326,103279.0,219113,2075252,Europe,60ft Dolls,The Big 3,1996,41.67,13,['Album'],['Rock'],"['Britpop', 'Indie Rock', 'Hard Rock']","Rock trio from Newport, South Wales, in the st..."
2899,14085388.0,3566297,31387334,Europe,86TVs,86TVs,2024,0.00,15,"['LP', 'Album', 'Limited Edition']","['Rock', 'Pop']","['Alternative Rock', 'Indie Rock']","Made up of [a499923] Felix and Hugo White, alo..."
6116,271475.0,424802,3562481,Europe,Accept,Stalingrad (Brothers In Death),2012,57.20,11,"['LP', 'Album']",['Rock'],['Heavy Metal'],German heavy metal band from the town of Solin...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2070,37757.0,11134,322188,Europe,a-ha,Hunting High And Low,1985,36.65,10,"['LP', 'Album', 'Stereo']","['Electronic', 'Pop']",['Synth-pop'],Norwegian band formed in Oslo in 1982. The ban...
2447,1971822.0,3117765,27255492,Europe,a.P.A.t.T.,We,2023,0.00,13,['Album'],"['Electronic', 'Rock']","['Avantgarde', 'Experimental', 'Prog Rock', 'P...",a.P.A.t.T. have been baffling audiences for ne...
3781,6774153.0,3038081,26603456,Europe,boygenius,The Record,2023,42.32,12,['Album'],['Rock'],"['Folk Rock', 'Indie Rock']","Phoebe Bridgers, Lucy Dacus and Julien Baker f..."
1088,3971017.0,725754,6023794,Europe,frnkiero andthe cellabration.,.Stomachaches.,2014,0.00,12,['Album'],['Rock'],[],NaN


In [15]:
percentage_album_length_missing = round((df_masters_concat['album_length']==0).sum() / df_masters_concat.shape[0] * 100, 2)
print(f'% missing album_length: {percentage_album_length_missing}%')

% missing album_length: 43.56%


In [16]:
df_masters_concat['artist_profile'].isna().sum()

1003

In [17]:
percentage_artist_profile_missing = round(df_masters_concat['artist_profile'].isna().sum() / df_masters_concat.shape[0] * 100, 2)
print(f'% missing artist_profile: {percentage_artist_profile_missing}%')

% missing artist_profile: 13.45%


In [18]:
df_masters_concat[df_masters_concat['artist_profile']==''] # should be empty

,artist_id,master_id,main_release_id,release_country,artist,title,year,album_length,tracks,release_type,genres,styles,artist_profile


In [19]:
df_masters_concat.loc[10, 'artist_profile']

'English Pop punk band from Southend-on-Sea, Essex, consisting of James Bourne, Matt Willis and Charlie Simpson. Formed in 2000, the band had four UK number-one singles, won two Brit awards and have released four studio albums, selling in excess of five million records.\r\n\r\nIn 2003, Dutch releases of this band were released under the artist name "Bust" to avoid confusion with Dutch band [a493201] who had a hit back then with "We Want More".'

In [30]:
df_masters_concat.groupby(["release_country", 'year']).agg({'master_id': 'count'}).sort_values('master_id', ascending=False).astype(int).head(10)

master_id
release_country year           
UK              2023        547
                2024        428
                2014        277
                2015        232
                2020        142
US              2014         95
                2023         84
                2024         79
UK              2003         68
                1985         62